In [1]:
import os, sys, glob
import numpy as np
import pandas as pd

import time
import datetime

from joblib import Parallel, delayed
from sklearn.metrics import f1_score, log_loss, classification_report
from sklearn.model_selection import StratifiedKFold

import lightgbm as lgb
pd.set_option('max_rows', 5000)
pd.set_option('max_columns', 5000)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def read_data():
    path = 'data/molecule_open_data/'
    print('Reading train.csv file....')
    train = pd.read_csv(path+'candidate_train.csv')
    print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))
    print('Reading specs.csv file....')
    specs = pd.read_csv(path +'train_answer.csv')
    print('Specs.csv file have {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))
    train = train.merge(specs,how='left',on='id')
    print('Reading test.csv file....')
    test = pd.read_csv(path + 'candidate_val.csv')
    print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))
    print('Reading sample_submission.csv file....')
    sample_submission = pd.read_csv(path+'sample_submission.csv')
    print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return train, test, sample_submission

In [ ]:
train, test, sample_submission = read_data()

In [13]:
 def _reduce_mem_usage_(df, verbose=True):
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        start_mem = df.memory_usage().sum() / 1024**2
        for col in df.columns:
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
        end_mem = df.memory_usage().sum() / 1024**2
        if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
        return df

In [30]:
path = 'data/molecule_open_data/'

# train = pd.read_csv(path+'candidate_train.csv')
# print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))
# print('Reading specs.csv file....')
# specs = pd.read_csv(path +'train_answer.csv')
# print('Specs.csv file have {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))
train = train.merge(specs,how='left',on='id')
# del specs

In [16]:
del train
print('Reading test.csv file....')
test = pd.read_csv(path + 'candidate_val.csv')
print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))


Reading test.csv file....
Test.csv file have 26358 rows and 3178 columns


In [28]:
train.shape,specs.shape
specs.describe()
# train.drop(['p1','p2','p3','p4','p5','p6'],axis=1,inplace=True)

,p1,p2,p3,p4,p5,p6
count,79267.000000,79267.000000,79267.000000,79267.000000,79267.000000,79267.000000
mean,2.676003,-0.240130,0.011622,0.148895,-411.080342,-411.071868
std,1.433312,0.021765,0.046745,0.033103,39.978193,39.977972
min,0.000000,-0.396300,-0.172000,0.025540,-714.568061,-714.560153
25%,1.583900,-0.252500,-0.023400,0.125553,-437.893493,-437.885360
50%,2.490500,-0.241100,0.012600,0.148484,-416.833806,-416.825714
75%,3.623600,-0.228800,0.050000,0.171291,-387.041513,-387.032126
max,19.687100,-0.132500,0.117100,0.273396,-40.478930,-40.476062


In [14]:
# train = _reduce_mem_usage_(train)
# test = _reduce_mem_usage_(test)
# train.to_pickle('data/train.pkl')
# test.to_pickle('data/test.pkl')

Mem. usage decreased to 243.87 Mb (87.3% reduction)


In [31]:
train[['p1','p2','p3','p4','p5','p6']].describe()

,p1,p2,p3,p4,p5,p6
count,79267.000000,79267.000000,79267.000000,79267.000000,79267.000000,79267.000000
mean,2.676003,-0.240130,0.011622,0.148895,-411.080342,-411.071868
std,1.433312,0.021765,0.046745,0.033103,39.978193,39.977972
min,0.000000,-0.396300,-0.172000,0.025540,-714.568061,-714.560153
25%,1.583900,-0.252500,-0.023400,0.125553,-437.893493,-437.885360
50%,2.490500,-0.241100,0.012600,0.148484,-416.833806,-416.825714
75%,3.623600,-0.228800,0.050000,0.171291,-387.041513,-387.032126
max,19.687100,-0.132500,0.117100,0.273396,-40.478930,-40.476062


In [32]:
# train = _reduce_mem_usage_(train)
# test = _reduce_mem_usage_(test)
train.to_pickle('data/train.pkl')
# test.to_pickle('data/test.pkl')

## 开始建模

In [1]:
import re
import gc
import pickle  
import random
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
# import tensorflow_hub as hub
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Input, Dropout, Lambda
from keras.optimizers import Adam
from keras.callbacks import Callback
# from scipy.stats import spearmanr, rankdata
# from os.path import join as path_join
from numpy.random import seed
# from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import MultiTaskElasticNet,MultiTaskLasso

Using TensorFlow backend.


In [2]:
train = pd.read_pickle('data/train.pkl')
test = pd.read_pickle('data/test.pkl')

In [3]:
# train[labels].describe()
features = []
for col in train.columns:
    val_len = len(list(set(train[col])))
    if val_len > 1:
        features.append(col)
        
labels = ['p1','p2','p3','p4','p5','p6']
drop_features = ['0','1','2','3','4','5']
features = [x for x in features if x not in labels+drop_features+['id']]

In [62]:
train.describe()

In [15]:
test.describe()

0        1        2        3        4        5             6  \
count  26358.0  26358.0  26358.0  26358.0  26358.0  26358.0  26358.000000   
mean       0.0      0.0      0.0      0.0      0.0      0.0      0.868010   
std        0.0      0.0      0.0      0.0      0.0      0.0      0.875178   
min        0.0      0.0      0.0      0.0      0.0      0.0      0.000000   
25%        0.0      0.0      0.0      0.0      0.0      0.0      0.000000   
50%        0.0      0.0      0.0      0.0      0.0      0.0      1.000000   
75%        0.0      0.0      0.0      0.0      0.0      0.0      1.000000   
max        0.0      0.0      0.0      0.0      0.0      0.0      6.000000   

                  7             8             9            10            11  \
count  26358.000000  26358.000000  26358.000000  26358.000000  26358.000000   
mean       0.000569      1.680932      0.108847      0.401472      0.213142   
std        0.023849      1.235987      0.328296      0.737549      0.643392   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      1.000000      0.000000      0.000000      0.000000   
50%        0.000000      2.000000      0.000000      0.000000      0.000000   
75%        0.000000      2.000000      0.000000      1.000000      0.000000   
max        1.000000      7.000000      3.000000      8.000000      6.000000   

                 12            13            14            15            16  \
count  26358.000000  26358.000000  26358.000000  26358.000000  26358.000000   
mean       1.495030      0.000607      0.316109      0.412361      0.231239   
std        1.356494      0.024631      0.638586      0.620411      0.634934   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        1.000000      0.000000      0.000000      0.000000      0.000000   
75%        2.000000      0.000000      0.000000      1.000000      0.000000   
max        8.000000      1.000000      6.000000      4.000000      4.000000   

                 17            18       19            20       21  \
count  26358.000000  26358.000000  26358.0  26358.000000  26358.0   
mean       0.009750      0.468511      0.0      0.094620      0.0   
std        0.133323      0.616678      0.0      0.314804      0.0   
min        0.000000      0.000000      0.0      0.000000      0.0   
25%        0.000000      0.000000      0.0      0.000000      0.0   
50%        0.000000      0.000000      0.0      0.000000      0.0   
75%        0.000000      1.000000      0.0      0.000000      0.0   
max        2.000000      3.000000      0.0      3.000000      0.0   

                 22            23            24            25       26  \
count  26358.000000  26358.000000  26358.000000  26358.000000  26358.0   
mean       0.067570      0.204340      0.054594      0.128955      0.0   
std        0.258017      0.438043      0.238754      0.353019      0.0   
min        0.000000      0.000000      0.000000      0.000000      0.0   
25%        0.000000      0.000000      0.000000      0.000000      0.0   
50%        0.000000      0.000000      0.000000      0.000000      0.0   
75%        0.000000      0.000000      0.000000      0.000000      0.0   
max        3.000000      3.000000      3.000000      3.000000      0.0   

                 27            28            29            30            31  \
count  26358.000000  26358.000000  26358.000000  26358.000000  26358.000000   
mean       0.084035      0.155285      0.159838      0.000986      0.031717   
std        0.298271      0.493130      0.371909      0.031392      0.177615   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      

In [4]:
X_train = train[features].values
X_test = test[features].values
y_train = train[labels].values
X_train.shape,y_train.shape,X_test.shape

((79267, 3126), (79267, 6), (26358, 3126))

In [36]:
# kf = KFold(n_splits=5, random_state=42, shuffle=True)
# for ind, (tr, val) in enumerate(kf.split(X_train)):
#     X_tr = X_train[tr]
#     y_tr = y_train[tr]
#     X_vl = X_train[val]
#     y_vl = y_train[val]
    
#     model = create_model()
#     model.fit(
#         X_tr, y_tr, epochs=100, batch_size=32, validation_data=(X_vl, y_vl), verbose=True, 
#         callbacks=[SpearmanRhoCallback(training_data=(X_tr, y_tr), validation_data=(X_vl, y_vl),
#                                        patience=5, model_name=u'best_model_batch.h5')]
#     )
#     model.load_weights('best_model_batch.h5')
#     all_predictions.append(model.predict(X_test))
    
#     os.remove('best_model_batch.h5')
    
# model = create_model()
# model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=False)
# all_predictions.append(model.predict(X_test))
    
# kf = KFold(n_splits=5, random_state=2019, shuffle=True)
# for ind, (tr, val) in enumerate(kf.split(X_train)):
#     X_tr = X_train[tr]
#     y_tr = y_train[tr]
#     X_vl = X_train[val]
#     y_vl = y_train[val]
#     model = MultiTaskElasticNet(alpha=0.001, random_state=42, l1_ratio=0.5)
#     model.fit(X_tr, y_tr)
#     all_predictions.append(model.predict(X_test))
    
model = MultiTaskElasticNet(alpha=0.0005, random_state=42, l1_ratio=0.3,max_iter=200,normalize=True, tol=0.001)
model.fit(X_train, y_train)
all_predictions = model.predict(X_test)

In [5]:
model = MultiTaskElasticNet(alpha=0.0001, random_state=42, l1_ratio=0.5)
model.fit(X_train, y_train)
all_predictions = model.predict(X_test)

C:\Users\Joymi\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1803: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 136784.61175083838, tolerance: 25356.348677257814
  check_random_state(self.random_state), random)


In [6]:
y_pre_trn = model.predict(X_train)
def smape(y_true, y_pred):
    return  2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100
print(smape(y_train, y_pre_trn))

In [24]:
path = 'data/molecule_open_data/'
targets = ['p1','p2','p3','p4','p5','p6']

submission = pd.read_csv(path+'sample_submission.csv')
print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
submission[targets] = all_predictions
submission.to_csv("data/submission.csv", index = False)
submission.head()

Sample_submission.csv file have 26358 rows and 7 columns


,id,p1,p2,p3,p4,p5,p6
0,fb63e856,5.083128,-0.274294,-0.072603,0.084621,-411.806269,-411.806188
1,8e928fd5,3.012934,-0.231633,-0.008907,0.149504,-402.252593,-402.243339
2,c3d2844a,2.544541,-0.243216,0.006724,0.119647,-396.111430,-396.109847
3,4470fabd,1.508454,-0.250974,0.075795,0.196929,-387.002656,-386.983081
4,9a246e47,3.229828,-0.271433,0.027688,0.193970,-367.329798,-367.318529


In [26]:
submission.describe()

,p1,p2,p3,p4,p5,p6
count,26358.000000,26358.000000,26358.000000,26358.000000,26358.000000,26358.000000
mean,2.668644,-0.240180,0.011729,0.149170,-411.205955,-411.198226
std,1.019081,0.018780,0.043870,0.032968,39.723267,39.723142
min,-0.137352,-0.332740,-0.174092,0.014068,-604.213507,-604.205758
25%,1.848979,-0.250448,-0.017872,0.125744,-437.982620,-437.975320
50%,2.603466,-0.241019,0.014535,0.148661,-417.229027,-417.222756
75%,3.418246,-0.230428,0.048729,0.171315,-386.895058,-386.888158
max,6.624848,-0.148263,0.103938,0.273059,-94.224636,-94.227477


## MultiOutputRegressor

In [5]:
X_train = train[features].values
X_test = test[features].values
y_train = train[labels].fillna(-999).values
X_train.shape,y_train.shape,X_test.shape

((79267, 3126), (79267, 6), (26358, 3126))

In [35]:
train[labels].describe()

,p1,p2,p3,p4,p5,p6
count,79267.000000,79267.000000,79267.000000,79267.000000,79267.000000,79267.000000
mean,2.676003,-0.240130,0.011622,0.148895,-411.080342,-411.071868
std,1.433312,0.021765,0.046745,0.033103,39.978193,39.977972
min,0.000000,-0.396300,-0.172000,0.025540,-714.568061,-714.560153
25%,1.583900,-0.252500,-0.023400,0.125553,-437.893493,-437.885360
50%,2.490500,-0.241100,0.012600,0.148484,-416.833806,-416.825714
75%,3.623600,-0.228800,0.050000,0.171291,-387.041513,-387.032126
max,19.687100,-0.132500,0.117100,0.273396,-40.478930,-40.476062


In [29]:
y_train[0]

array([ 1.181e+00, -2.437e-01,  5.759e-02,  1.703e-01, -4.402e+02,
       -4.402e+02], dtype=float16)

In [7]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

# MultiOutputRegressor(estimator=LGBMRegressor(boosting_type='gbdt',
#                                              class_weight=None,
#                                              colsample_bytree=1.0,
#                                              importance_type='split',
#                                              learning_rate=0.1, max_depth=-1,
#                                              min_child_samples=20,
#                                              min_child_weight=0.001,
#                                              min_split_gain=0.0,
#                                              n_estimators=100, n_jobs=-1,
#                                              num_leaves=31, objective=None,
#                                              random_state=None, reg_alpha=0.0,
#                                              reg_lambda=0.0, silent=True,
#                                               subsample=1.0,
#                                              subsample_for_bin=200000,
#                                              subsample_freq=0),
#                      n_jobs=None)

In [79]:
single_model = lgb.LGBMRegressor(objective='regression', num_leaves=31,arning_rate=0.05, n_estimators=1000,random_state=88,n_jobs=-1)
model = MultiOutputRegressor(single_model)
# fit_param = {'verbose': False, 'early_stopping_rounds':50}#, 'eval_set':(eval_X, eval_y)}
model.fit(X_train, y_train)#, fit_param=fit_param)
y_pre_trn = model.predict(X_train)

In [80]:
def smape(y_true, y_pred):
    return  2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100
print(smape(y_train, y_pre_trn))

7.8485459581725845


In [13]:

all_predictions = model.predict(X_test)
path = 'data/molecule_open_data/'
targets = ['p1','p2','p3','p4','p5','p6']
submission = pd.read_csv(path+'sample_submission.csv')
print('Sample_submission.csv file have {} rows and {} columns'.format(submission.shape[0], submission.shape[1]))
sub = test[['id','3176']].merge(submission,how='left',on='id')
sub[targets] = all_predictions
sub[['id']+targets].to_csv("data/submission.csv", index = False)
# submission[targets] = all_predictions
# submission.to_csv("data/submission.csv", index = False)
# submission.head()

NameError: name 'sub' is not defined

In [8]:
import catboost as cab

clf=cab.CatBoostRegressor(iterations=750
                      ,learning_rate=0.05
                      ,depth=5
                      ,loss_function='RMSE'
                      ,silent=True
                      ,gpu_cat_features_storage=-1)
model = MultiOutputRegressor(clf)
model.fit(X_train,y_train)

MultiOutputRegressor(estimator=<catboost.core.CatBoostRegressor object at 0x000001279130BA88>,
                     n_jobs=None)

In [10]:
y_pre_trn = model.predict(X_train)
def smape(y_true, y_pred):
    return  2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100
print(smape(y_train, y_pre_trn))

In [12]:
all_predictions = model.predict(X_test)

In [15]:
path = 'data/molecule_open_data/'
targets = ['p1','p2','p3','p4','p5','p6']
submission = pd.read_csv(path+'sample_submission.csv')
print('Sample_submission.csv file have {} rows and {} columns'.format(submission.shape[0], submission.shape[1]))
sub = test[['id','3176']].merge(submission,how='left',on='id')
sub[targets] = all_predictions
sub[['id']+targets].to_csv("data/submission.csv", index = False)

Sample_submission.csv file have 26358 rows and 7 columns


In [ ]:
rf = RandomForestRegressor(n_estimators=100,max_depth=10,random_state=0)
model = MultiOutputRegressor(rf).fit(X_train, y_train)
y_pre_trn = model.predict(X_train)

In [ ]:
xlf=xgb.XGBClassifier(max_depth=6
                      ,learning_rate=0.01
                      ,n_estimators=1000
#                       ,reg_alpha=0.004
                      ,n_jobs=-1
#                       ,reg_lambda=0.002
                      ,importance_type='total_cover')
model = MultiOutputRegressor(xlf).fit(X_train, y_train) 
# y_pre_trn = model.predict(X_train)

In [15]:
def smape(y_true, y_pred):
    return  np.mean(2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100)
print(smape(y_train, y_pre_trn))

10.010128038662405


In [ ]:
1

In [ ]:
all_predictions = model.predict(X_test)

path = 'data/molecule_open_data/'
targets = ['p1','p2','p3','p4','p5','p6']

submission = pd.read_csv(path+'sample_submission.csv')
print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
submission[targets] = all_predictions
submission.to_csv("data/submission.csv", index = False)
submission.head()

## 深度学习

In [5]:
path = 'data/molecule_open_data/'

specs = pd.read_csv(path +'train_answer.csv')
train2 = train[features+['id']].merge(spec01s,how='inner',on='id')

In [6]:
# train['p1'].map(lambda x: np.log(1+x)).describe()
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
ss = MinMaxScaler()
train2[labels] = ss.fit_transform(train2[labels])

In [7]:
train2[labels].describe()

,p1,p2,p3,p4,p5,p6
count,79267.000000,79267.000000,79267.000000,79267.000000,79267.000000,79267.000000
mean,0.135927,0.592003,0.635151,0.497687,0.450219,0.450223
std,0.072805,0.082508,0.161691,0.133558,0.059307,0.059307
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.080454,0.545110,0.514009,0.403513,0.410442,0.410446
50%,0.126504,0.588324,0.638533,0.496030,0.441684,0.441687
75%,0.184060,0.634951,0.767900,0.588047,0.485880,0.485886
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
X_train = train2[features].values
X_test = test[features].values
y_train = train2[labels].values
X_train.shape,y_train.shape,X_test.shape

((79267, 3177), (79267, 6), (26358, 3177))

In [84]:
# Compatible with tensorflow backend
def smape(y_true, y_pred):
    return  2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100
class SpearmanRhoCallback(Callback):
    def __init__(self, training_data, validation_data, patience, model_name):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        
        self.patience = patience
        self.value = -1
        self.bad_epochs = 0
        self.model_name = model_name

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return
    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.x_val)
#         rho_val = np.mean([spearmanr(self.y_val[:, ind], y_pred_val[:, ind] + np.random.normal(0, 1e-7, y_pred_val.shape[0])).correlation for ind in range(y_pred_val.shape[1])])
        rho_val = smape(self.y_val,y_pred_val)
        if rho_val >= self.value:
            self.value = rho_val
        else:
            self.bad_epochs += 1
        if self.bad_epochs >= self.patience:
            print("Epoch %05d: early stopping Threshold" % epoch)
            self.model.stop_training = True
            #self.model.save_weights(self.model_name)
        print('\rval_spearman-rho: %s' % (str(round(rho_val, 4))), end=100*' '+'\n')
        return rho_val

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [90]:
def create_model():
    inps = Input(shape=(X_train.shape[1],))
    x = Dense(512, activation='elu')(inps)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='elu')(inps)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='elu')(inps)
    x = Dropout(0.2)(x)
    x = Dense(y_train.shape[1])(x)
    model = Model(inputs=inps, outputs=x)
    model.compile(
        optimizer=Adam(lr=1e-3),
        loss=['mse'],
        metrics=['mse']
    )
    model.summary()
    return model

In [91]:
all_predictions = []
kf = KFold(n_splits=5, random_state=42, shuffle=True)
for ind, (tr, val) in enumerate(kf.split(X_train)):
    X_tr = X_train[tr]
    y_tr = y_train[tr]
    X_vl = X_train[val]
    y_vl = y_train[val]
    
    model = create_model()
    model.fit(
        X_tr, y_tr, epochs=100, batch_size=512, validation_data=(X_vl, y_vl), verbose=True, 
        callbacks=[SpearmanRhoCallback(training_data=(X_tr, y_tr), validation_data=(X_vl, y_vl),
                                       patience=5, model_name=f'best_model_batch{ind}.h5')]
    )
    all_predictions.append(model.predict(X_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 3126)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 128)               400256    
_________________________________________________________________
dropout_21 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 6)                 774       
Total params: 401,030
Trainable params: 401,030
Non-trainable params: 0
_________________________________________________________________
Train on 63413 samples, validate on 15854 samples
Epoch 1/100
63413/63413 [==============================] - 6s 101us/step - loss: 7950.5594 - mean_squared_error: 7950.5594 - val_loss: 826.7035 - val_mean_squared_error: 826.7035
val_spearman-rho: 91.9549              

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 3126)              0         
_________________________________________________________________
dense_39 (Dense)             (None, 128)               400256    
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 6)                 774       
Total params: 401,030
Trainable params: 401,030
Non-trainable params: 0
_________________________________________________________________
Train on 63414 samples, validate on 15853 samples
Epoch 1/100
63414/63414 [==============================] - 6s 102us/step - loss: 9338.2715 - mean_squared_error: 9338.2715 - val_loss: 868.9782 - val_mean_squared_error: 868.9782
val_spearman-rho: 107.1092             

In [92]:
test_preds = np.array([np.array([c for c in p.T]).T for p in all_predictions]).mean(axis=0)
# max_val = test_preds.max() + 1
# test_preds = test_preds/max_val + 1e-12

In [98]:
path = 'data/molecule_open_data/'
targets = ['p1','p2','p3','p4','p5','p6']
all_predictions = model.predict(X_test)

submission = pd.read_csv(path+'sample_submission.csv')
print('Sample_submission.csv file have {} rows and {} columns'.format(submission.shape[0], submission.shape[1]))
submission = test[['id','3176']].merge(submission,how='left',on='id')
submission[targets] = test_preds
submission[['id']+targets].to_csv("data/submission.csv", index = False)
# submission[targets] = all_predictions
# submission.to_csv("data/submission.csv", index = False)
# submission.head()

Sample_submission.csv file have 26358 rows and 7 columns


In [102]:
submission[['id']+targets].head()

,id,p1,p2,p3,p4,p5,p6
0,66b19220,2.789745,-0.291671,0.003310,0.364396,-404.231201,-407.084656
1,bed4af38,2.547901,-0.237029,-0.034049,0.110906,-392.920502,-393.223694
2,96f5658c,2.619061,-0.249644,0.003298,0.145408,-391.041809,-392.493073
3,d7031639,2.497406,-0.175145,-0.025903,0.128685,-372.197937,-373.522491
4,1fdd8f3c,2.415227,-0.201413,0.076055,-0.009157,-376.351624,-375.182129


In [103]:
sub[['id']+targets].head()

,id,p1,p2,p3,p4,p5,p6
0,66b19220,3.800251,-0.244197,-0.079438,0.085117,-410.674425,-410.948315
1,bed4af38,3.739651,-0.232559,0.008596,0.148867,-401.998657,-401.990540
2,96f5658c,1.998464,-0.239771,0.004678,0.118948,-396.297152,-396.330058
3,d7031639,1.881109,-0.246075,0.075563,0.195995,-387.425384,-387.410067
4,1fdd8f3c,4.026430,-0.295697,0.030353,0.194003,-367.012282,-367.078586


In [16]:
sub[['id']+targets].head()

,id,p1,p2,p3,p4,p5,p6
0,66b19220,4.062343,-0.263392,-0.078770,0.084936,-410.041282,-409.560534
1,bed4af38,3.581449,-0.230359,0.002486,0.149772,-401.843063,-401.815817
2,96f5658c,2.209289,-0.234245,0.009137,0.120337,-397.200460,-397.422233
3,d7031639,1.808722,-0.246699,0.080729,0.196816,-387.863014,-388.225345
4,1fdd8f3c,4.031777,-0.291672,0.035315,0.194299,-367.282490,-367.336377


In [106]:
submission[['id']+targets].to_csv("data/submission.csv", index = False)